In [69]:
from processor import *
from utils3 import *
from utils2 import *
import pandas as pd
import os
import sys
import functools
import operator
import random
from tqdm import tqdm, trange
import time
import numpy as np
import pandas
from math import degrees
import requests as r
from Bio import SeqIO, pairwise2
from io import StringIO
import gemmi
from gemmi import cif

In [5]:
PATH = 'data/'
STRUCTURE_PATH = PATH + 'mmcif/'
"""
self.remove_hetatm = remove_hetatm
self.allow_exception = allow_exception

self.filenames, self.pdb_ids = self.get_pdb_files()
self.filenames = self.filenames[starting_idx:]
self.pdb_ids = self.pdb_ids[starting_idx:]
if self.limit == None:
    self.limit = len(self.pdb_ids)
if len(self.filenames) > self.limit:
    self.filenames = self.filenames[:self.limit]
    self.pdb_ids = self.pdb_ids[:self.limit]"""

# Columns for structure dataframe
COLS = ['group_PDB', 'auth_asym_id', 'label_asym_id', 'label_seq_id', 'auth_seq_id', 
             'label_comp_id', 'id', 'label_atom_id', 
             'type_symbol', 'Cartn_x', 'Cartn_y', 'Cartn_z']

In [147]:
ALL_COLS = ['group_PDB' ,
'id' ,
'type_symbol' ,
'label_atom_id' ,
'label_alt_id' ,
'label_comp_id' ,
'label_asym_id' ,
'label_entity_id' ,
'label_seq_id' ,
'pdbx_PDB_ins_code' ,
'Cartn_x' ,
'Cartn_y' ,
'Cartn_z' ,
'occupancy' ,
'B_iso_or_equiv' ,
'pdbx_formal_charge' ,
'auth_seq_id' ,
'auth_comp_id' ,
'auth_asym_id' ,
'auth_atom_id' ,
'pdbx_PDB_model_num' ]

In [11]:
def load_cifs(pdb_id):
    path = 'data/mmcif/' + pdb_id + '.cif'
    try:
        doc = cif.read_file(path)  # copy all the data from mmCIF file
        lol = []  # list of lists
        for b, block in enumerate(doc):
            table = block.find('_atom_site.', COLS)
            for row in table:
                lol.append([pdb_id]+list(row))

    except Exception as e:
        print("Hoppla. %s" % e)
        sys.exit(1)
    cols = ['PDB']+COLS
    st = gemmi.read_structure(path)
    model = st[0]
    if len(st) > 1:
        print("There are multiple models!")
    rol = []
    for chain in model:
        for r, res in enumerate(chain.get_polymer()):
            # previous_residue() and next_residue() return previous/next
            # residue only if the residues are bonded. Otherwise -- None.
            prev_res = chain.previous_residue(res)
            next_res = chain.next_residue(res)
            try:
                phi, psi = gemmi.calculate_phi_psi(prev_res, res, next_res)
            except:
                phi, psi = np.nan, np.nan
            try:
                omega = gemmi.calculate_omega(res, next_res)
            except:
                omega = np.nan
            rol.append([res.label_seq, res.subchain, 
                        degrees(phi), degrees(omega), degrees(psi)])
    cols2 = ['label_seq_id', 'label_asym_id', 'phi', 'omega', 'psi']
    rol_df = pd.DataFrame(data=rol, columns=cols2)
    rol_df['label_seq_id'] = rol_df['label_seq_id'].astype(int)
    lol_df = pd.DataFrame(data=lol, columns=cols)
    lol_df['label_seq_id'] = lol_df.apply(lambda x: int(x.label_seq_id) if x.label_seq_id != '.' else np.nan, axis=1)
    return pd.merge(lol_df, rol_df, how='inner', on=['label_asym_id', 'label_seq_id'])

In [193]:
p = CifProcessor()
p.read_pkl_metainfo()
p.read_pkl(mode='rg', folder='data/processed/')

Reading files with generic numbers on receptors and gproteins.


In [203]:
f_act = p.filter_dfl_via_table(State='Active')

In [204]:
p.apply_filter(f_act)

In [227]:
f_act.head(20)

,uniprot(gene),receptor family,Cl.,Species,Method,PDB,Resolution,Preferred Chain,State,Degree active %,Family,Subtype,Function
0,STE2,STE2,D1(Ste2-likefungalpheromone),strainATCC204508/S288c,cryo-EM,7AD3,3.5,A,Active,-,GPa1family,αPa1,Agonist
1,5HT1B,5-Hydroxytryptamine,A(Rhodopsin),Human,cryo-EM,6G79,3.8,S,Active,100,Gi/o,αo,Agonist
7,CXCR2,Chemokine,A(Rhodopsin),Human,cryo-EM,6LFO,3.4,R,Active,100,Gi/o,αi1,Agonist
8,CXCR2,Chemokine,A(Rhodopsin),Human,cryo-EM,6LFM,3.5,R,Active,100,Gi/o,αi1,Agonist
12,ADA2B,Adrenoceptors,A(Rhodopsin),Human,cryo-EM,6K42,4.1,R,Active,100,Gi/o,αi1,Agonist
13,ADA2B,Adrenoceptors,A(Rhodopsin),Human,cryo-EM,6K41,2.9,R,Active,100,Gi/o,αo,Agonist
22,PE2R3,Prostanoid,A(Rhodopsin),Human,X-ray,6AK3,2.9,B,Active,85,-,-,Agonist
24,NTR1,Neurotensin,A(Rhodopsin),Human,cryo-EM,6OSA,3.0,R,Active,100,Gi/o,αi1,Agonist
25,NTR1,Neurotensin,A(Rhodopsin),Human,cryo-EM,6OS9,3.0,R,Active,100,Gi/o,αi1,Agonist
28,PTH1R,Parathyroidhormone,B1(Secretin),Human,cryo-EM,6NBI,4.0,R,Active,100,Gs,αs,Agonist


In [232]:
structure = p.dfl[p.dfl_list.index('7JVR')]

In [234]:
structure[structure['gprot_pos']!='']

,PDB,group_PDB,auth_asym_id,label_asym_id,label_seq_id,auth_seq_id,label_comp_id,id,label_atom_id,type_symbol,...,identifier,label_comp_sid,label_2_uni,gen_pos,gen_pos1,gen_pos2,uniprot_comp_sid,gprot_pos,uniprot_comp_id,fam_comp_id
2165,7JVR,ATOM,A,B,5,5,LEU,2166,CA,C,...,DRD2_HUMAN,L,4,,0.0,0,,G.HN.11,L,L
2173,7JVR,ATOM,A,B,6,6,SER,2174,CA,C,...,DRD2_HUMAN,S,5,,0.0,0,,G.HN.30,S,S
2179,7JVR,ATOM,A,B,7,7,ALA,2180,CA,C,...,DRD2_HUMAN,A,6,,0.0,0,,G.HN.31,A,A
2184,7JVR,ATOM,A,B,8,8,GLU,2185,CA,C,...,DRD2_HUMAN,E,7,,0.0,0,,G.HN.32,E,E
2193,7JVR,ATOM,A,B,9,9,ASP,2194,CA,C,...,DRD2_HUMAN,D,8,,0.0,0,,G.HN.33,D,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3934,7JVR,ATOM,A,B,350,350,ASP,3935,CA,C,...,DRD2_HUMAN,D,349,,0.0,0,,G.H5.22,D,D
3942,7JVR,ATOM,A,B,351,351,CYS,3943,CA,C,...,DRD2_HUMAN,C,350,,0.0,0,,G.H5.23,C,C
3948,7JVR,ATOM,A,B,352,352,GLY,3949,CA,C,...,DRD2_HUMAN,G,351,,0.0,0,,G.H5.24,G,G
3952,7JVR,ATOM,A,B,353,353,LEU,3953,CA,C,...,DRD2_HUMAN,L,352,,0.0,0,,G.H5.25,L,L


In [235]:
def spread_gen_pos_to_residue_atoms(structure):
    if 'gprot_pos' in list(structure.columns):
        structure['generic_position'] = structure.apply(lambda x: x.gen_pos if x.gprot_pos == '' else x.gprot_pos, axis=1)
        c_gprot = list(structure[structure['gprot_pos'] != '']['label_asym_id'])
    else:
        structure['generic_position'] = structure['gen_pos']
        c_gprot = []
    residue_seq_ids = list(set(list(structure['label_seq_id'])))
    chains = list(set(list(structure['label_asym_id'])))
    # for all chains
    for c in tqdm(chains):
        c_rec = list(structure[structure['gen_pos'] != '']['label_asym_id'])
        check_gen_pos_chains = list(set(c_rec+c_gprot))
        # check if chain has residues with a generic position number
        if c in check_gen_pos_chains:
            # for all residues
            for rsi in tqdm(residue_seq_ids):
                block_gen_pos_ = structure[(structure['label_seq_id']==rsi) &
                                           (structure['label_asym_id']==c) &
                                           (structure['generic_position']!='')]
                if len(block_gen_pos_)>0: 
                    block_idxs = list(structure[(structure['label_seq_id']==rsi) &
                                                (structure['label_asym_id']==c)].index)
                    block_gen_pos = block_gen_pos_['generic_position'].iloc[0]
                    # for all indexes within that residue
                    for idx in block_idxs:
                        structure.loc[idx, 'generic_position'] = block_gen_pos
    return structure

In [236]:
df = spread_gen_pos_to_residue_atoms(structure)

 87%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                         | 388/447 [00:04<00:00, 137.21it/s]


 93%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉             | 417/447 [00:03<00:00, 77.37it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.76s/it]


In [237]:
df[df['generic_position']!='']

,PDB,group_PDB,auth_asym_id,label_asym_id,label_seq_id,auth_seq_id,label_comp_id,id,label_atom_id,type_symbol,...,label_comp_sid,label_2_uni,gen_pos,gen_pos1,gen_pos2,uniprot_comp_sid,gprot_pos,uniprot_comp_id,fam_comp_id,generic_position
0,7JVR,ATOM,R,A,179,34,TYR,1,N,N,...,Y,0,,0.00,0,,,,,1.32x32
1,7JVR,ATOM,R,A,179,34,TYR,2,CA,C,...,Y,34,1.32x32,1.32,32,Y,,,,1.32x32
2,7JVR,ATOM,R,A,179,34,TYR,3,C,C,...,Y,0,,0.00,0,,,,,1.32x32
3,7JVR,ATOM,R,A,179,34,TYR,4,O,O,...,Y,0,,0.00,0,,,,,1.32x32
4,7JVR,ATOM,R,A,179,34,TYR,5,CB,C,...,Y,0,,0.00,0,,,,,1.32x32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3966,7JVR,ATOM,A,B,354,354,PHE,3967,CD2,C,...,F,0,,0.00,0,,,,,G.H5.26
3967,7JVR,ATOM,A,B,354,354,PHE,3968,CE1,C,...,F,0,,0.00,0,,,,,G.H5.26
3968,7JVR,ATOM,A,B,354,354,PHE,3969,CE2,C,...,F,0,,0.00,0,,,,,G.H5.26
3969,7JVR,ATOM,A,B,354,354,PHE,3970,CZ,C,...,F,0,,0.00,0,,,,,G.H5.26


In [238]:
def write_cif_with_gen_seq_numbers(structure, replace_idx=-4):
    pdb_id = structure['PDB'].iloc[0]
    path = 'data/mmcif/' + pdb_id + '.cif'
    doc = cif.read_file(path)  # copy all the data from mmCIF file
    lol = []  # list of lists
    for b, block in enumerate(doc):
        col_overwrite = block.find('_atom_site.', ['id'])
        table = block.find('_atom_site.', ALL_COLS)
        for r, row in enumerate(table):
            if row[1] in list(structure['id']):
                gen_pos = structure[structure['id']==row[1]]['generic_position'].iloc[0]
                if gen_pos == '':
                    gen_pos = '-'
                row[replace_idx] = gen_pos
            else:
                row[replace_idx] = '-'
    return doc

In [241]:
doc = write_cif_with_gen_seq_numbers(df, replace_idx = -7)

In [242]:
doc.write_file('7JVR_modified.cif')